### 0) Set up Revision

In [ ]:
import shutil
import os
import arcpy

# === Set version info ===
old_version = "10"
new_version = "11"

# === Define paths ===
base_path = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\_MCD"
folder_template = "005356185-{} - THO - Master Construction Drawing"

### 1) Copies the folder from old version to a new version


In [ ]:
src_folder = os.path.join(base_path, folder_template.format(old_version))
dst_folder = os.path.join(base_path, folder_template.format(new_version))

# === Step 1: Copy the folder ===
try:
    shutil.copytree(src_folder, dst_folder)
    print(f"✅ Folder copied from:\n{src_folder}\nto:\n{dst_folder}")
except FileExistsError:
    print(f"⚠️ Destination folder already exists:\n{dst_folder}")
 
except FileNotFoundError:
    print(f"❌ Source folder not found:\n{src_folder}")
  
except Exception as e:
    print(f"❌ Error copying folder:\n{e}")


### 2) Renames all files inside the copied folder 

In [ ]:
# Rename files inside the copied folder ===
renamed_files = 0
for root, dirs, files in os.walk(dst_folder):
    for filename in files:
        if f"-{old_version}" in filename:
            old_path = os.path.join(root, filename)
            new_filename = filename.replace(f"-{old_version}", f"-{new_version}")
            new_path = os.path.join(root, new_filename)
            os.rename(old_path, new_path)
            renamed_files += 1
            print(f"🔁 Renamed: {filename} → {new_filename}")

print(f"✅ Finished. {renamed_files} files renamed in folder version {new_version}.")

### 3) As Built Monopiles

- 🌐 Open: [https://rwe.vissim.no/flexview](https://rwe.vissim.no/flexview) to see the latest update, click the **"ℹ️" (Info)** icon in the lower-left corner for more information.

#### 📌 As Built Monopiles from JDN & AS BUILT: Monopile Outer Diameter – **8.6 m**

Please complete all steps **before running the script below**:

1. 🔽 **Download PDFs**  
   - URL: [https://share.jandenul.com/](https://share.jandenul.com/)  
   - **Username**: DKTRTI,  **Password**: RKHJEB5LvcnVZw2

2. 💾 **Save PDFs** here:  
   *\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250505 As Built Monopiles from JDN*


3. 📋 **Extract and copy relevant data** into this Excel file:  
   *\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250505 As Built Monopiles from JDN\DK_THO_AsBuilt_Monopile_pt_UTM32N_ta_v0.xlsx*


4. 📤 **Export the cleaned data to CSV**: 
   *\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250505 As Built Monopiles from JDN\DK_THO_AsBuilt_Monopile_pt_UTM32N_ta_v0.csv*


##### ⚠️ CRITICAL NOTE: Before Running the Code

- ❗ **Ensure that the `005356185_THOR_Master_Construction_Drawing.aprx` project is closed** before running the script — otherwise layer will be not in the content (run over different aprx or VS code) 

    Two files are ouput:
    - *DK_THO_AsBuilt_Monopile_pt_UTM32N_pt_v0*
    - *DK_THO_AsBuilt_Monopile_OuterDiameter8pt6m_py_UTM32N_pt_v0*
    

- ❗ After running the script, **open the attribute table** for:  
 * 2_FINAL\WTG\WTG.gdb\DK_THO_AsBuilt_Monopile_OuterDiameter8pt6m_py_UTM32N_pt_v0* **Delete any empty rows manually**



In [ ]:
# === INPUTS ===
csv_file = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250505 As Built Monopiles from JDN\DK_THO_AsBuilt_Monopile_pt_UTM32N_ta_v0.csv"
x_field = "POS_ASBUILT_EAST"
y_field = "POS_ASBUILT_NORTH"
spatial_ref = arcpy.SpatialReference(32632)  # UTM Zone 32N

# === OUTPUTS ===
point_fc = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\WTG\WTG.gdb\DK_THO_AsBuilt_Monopile_pt_UTM32N_pt_v0"
buffer_fc = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\WTG\WTG.gdb\DK_THO_AsBuilt_Monopile_OuterDiameter8pt6m_py_UTM32N_pt_v0"


# === STEP 1: Create Point Feature Class ===
if arcpy.Exists(point_fc):
    arcpy.Delete_management(point_fc)
    print(f"✅ Point feature class was deleted: {point_fc}")

arcpy.management.XYTableToPoint(
    in_table=csv_file,
    out_feature_class=point_fc,
    x_field=x_field,
    y_field=y_field,
    coordinate_system=spatial_ref
)

print(f"✅ Point feature class created: {point_fc}")

# === STEP 2: Create 4.3m Buffer ===
if arcpy.Exists(buffer_fc):
    arcpy.Delete_management(buffer_fc)
    print(f"✅ Point feature class was deleted: {buffer_fc}")

arcpy.analysis.PairwiseBuffer(
    in_features=point_fc,
    out_feature_class=buffer_fc,
    buffer_distance_or_field="4.3 Meters",
    dissolve_option="NONE"
)

print(f"✅ Buffer feature class created: {buffer_fc}")


### 4) ANNOTATION – As Built Monopile Location*

1. 🔁 **Copy & Rename the Map**  
   - Open the .aprx project and *Duplicate* the map named:  
     *005356185-{old_version} THOR Master Construction Drawing CAD*  
   - Rename the copy to:  
     *005356185-{new_version} THOR Master Construction Drawing CAD*`
     

2. 👁️ **Check Labels**  
   - Ensure labels are **visible** for the following feature class:  
     **As Built Monopile** - DK_THO_AsBuilt_Monopile_pt_UTM32N_pt_v0`
     

3. 💬 **Convert Labels to Annotation**  
   - If labels are showing correctly:  
     → Run the **"Convert Labels to Annotation"** tool  
     → Attribute table will be udpated automaticly with "ANNOTATION – As Built Monopile Location"
     → Save the annotation layer as: *DK_THO_AsBuilt_Monopile_pt_UTM32N_pt_v0_Annotation*
          

4. ➕ **Add to New Map**  
   - Add the newly created annotation layer to the map:  
     *005356185-{new_version} THOR Master Construction Drawing CAD*

---



In [ ]:
# === Load the project ===
import arcpy

# === INPUTS ===
new_version = "11"
aprx_path = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\_MCD\005356185_THOR_Master_Construction_Drawing.aprx"
output_gdb = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\WTG\WTG.gdb"
layer_name = "As Built Monopile"
output_annotation_suffix = "_Annotation"
new_map_name = f"005356185-{new_version} THOR Master Construction Drawing CAD"
reference_scale = 100

aprx = arcpy.mp.ArcGISProject(aprx_path)
target_map = next((m for m in aprx.listMaps() if m.name == new_map_name), None)
if not target_map:
    raise Exception(f"❌ Map '{new_map_name}' not found.")
print(f"✅ Map '{new_map_name}' loaded.")

# === Find the layer ===
layer = next((lyr for lyr in target_map.listLayers() if lyr.name == layer_name), None)
if not layer or not layer.supports("SHOWLABELS"):
    raise Exception(f"❌ Layer '{layer_name}' not found or doesn't support labels.")

layer.showLabels = True
print(f"✅ Labels enabled for '{layer_name}'.")


arcpy.cartography.ConvertLabelsToAnnotation(
    input_map= target_map,
    conversion_scale= 100,
    output_geodatabase= output_gdb,
    anno_suffix= output_annotation_suffix,
    extent='394631.907089481 6223623.59608526 450316.163918058 6267404.33424396 PROJCS["WGS_1984_UTM_Zone_32N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",9.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
    generate_unplaced="GENERATE_UNPLACED",
    require_symbol_id=None,
    feature_linked=None,
    auto_create="AUTO_CREATE",
    update_on_shape_change="SHAPE_UPDATE",
    output_group_layer="GroupAnno",
    which_layers="SINGLE_LAYER",
    single_layer="As Built Monopile",
    multiple_feature_classes="FEATURE_CLASS_PER_FEATURE_LAYER",
    merge_label_classes="NO_MERGE_LABEL_CLASS"
)


# Rename the feature class, if the n
annotation_fc = f"{output_gdb}/As_Built_Monopile{output_annotation_suffix}"
new_ann_fc = f"{output_gdb}/DK_THO_AsBuilt_Monopile_pt_UTM32N_pt_v0_Annotation"     

if arcpy.Exists(annotation_fc):
    print(f"✅ Annotation feature class saved at: {annotation_fc}")
    if arcpy.Exists(new_fc):
        arcpy.Delete_management(new_ann_fc)
    arcpy.Rename_management(annotation_fc, new_ann_fc)
    print(f"✅ Renamed annotation feature class to: {new_ann_fc}")
else:
    # Print warning if not found
    print(f"⚠️ Annotation feature class NOT found at: {annotation_fc}")
    
    

# Check if feature class exists
if arcpy.Exists(new_ann_fc):
    # Add 'Layer' field if it doesn't exist
    if "Layer" not in [f.name for f in arcpy.ListFields(new_ann_fc)]:
        arcpy.AddField_management(new_ann_fc, "Layer", "TEXT", field_length=255)
        print(f"➕ Added missing 'Layer' field to: {new_ann_fc}")
    else:
        print(f"ℹ️ 'Layer' field already exists in: {new_ann_fc}")
    count = 0
    with arcpy.da.UpdateCursor(new_ann_fc, ["Layer"]) as cursor:
        for row in cursor:
            if row[0] != "ANNOTATION – As Built Monopile Location":
                row[0] = "ANNOTATION – As Built Monopile Location"
                cursor.updateRow(row)
                count += 1

    print(f"✅ Updated {count} records in 'Layer' field of: {new_fc}")
else:
    print(f"⚠️ Feature class not found: {new_fc}")
    
print("✅✅ Layer field values updated **ANNOTATION – As Built Monopile Location** successfully.")    
    

### 5) SST (VESSEL – As Built Footprint – MPI Adventure)

- **Step 1:** Open **EcoCod** and search for the title: *'VAO - Adventure T*'*

- **Step 2:** Review the top search results, specifically project 005938436-01.

- **Step 3:** The relevant DWG file from ecodoc should be saved to:  
  \\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250604 As Built Secondary Steel Installation from Van Oord
  
  It will come with the improve once data available


### 6) Add ANY LAYER  to MCD

1. **Add Fields**
   - Add a new field called *LAYER* (*String*) to the feature class.
   - Add a new field called *SYMBOL* (*String*) to the feature class.
   

2. **Update Field Values**
   - Populate the *LAYER* field with values from the corresponding *cad_layer_name*.
   - Populate the *SYMBOL* field with values from the corresponding *ArcGIS_value*.
   

3. **Delete the *Entity* Field**
   - Remove the *Entity* field from the feature class.
   - This field caused issues when exporting the feature class to CAD, specifically geometry errors.


#### ✅ Section 1: Update LAYER Field

In [ ]:
import arcpy

# === Path to feature class ===
gdb_path = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\VESSELS\VESSELS.gdb"
name_FC = "DK_THO_JB115_JackUpBarge_pt_UTM32N_v0"
fc = f"{gdb_path}\\{name_FC}"

# === Add LAYER field if missing ===
if "Layer" not in [f.name for f in arcpy.ListFields(fc)]:
    arcpy.AddField_management(fc, "LAYER", "TEXT")
    print("🆕 Field 'LAYER' added.")
else:
    print("✅ Field 'LAYER' already exists.")

# === Layer mapping ===
layer_map = {
    "JUB Install":               "VESSEL - As Planned JUB Mooring Plan Installation", 
    "JUB Stand Off":             "VESSEL - As Planned JUB Mooring Plan Standoff",
    "Mooring wire Installation": "VESSEL - As Planned JUB Mooring Plan Installation",
    "Mooring wire Stand Off":    "VESSEL - As Planned JUB Mooring Plan Standoff",
}

# === Update LAYER field ===
with arcpy.da.UpdateCursor(fc, ["LAYER"]) as cursor:
    for row in cursor:
        original = row[0]
        row[0] = layer_map.get(row[0], row[0])
        if row[0] != original:
            print(f"🔁 LAYER updated: '{original}' ➝ '{row[0]}'")
        cursor.updateRow(row)

print("✅ Section 1 complete: LAYER field updated.")


#### ✅ Section 2: Update SYMBOL Field

In [ ]:
# === Add SYMBOL field if missing ===
if "SYMBOL" not in [f.name for f in arcpy.ListFields(fc)]:
    arcpy.AddField_management(fc, "SYMBOL", "TEXT")
    print("🆕 Field 'SYMBOL' added.")
else:
    print("✅ Field 'SYMBOL' already exists.")

# === Symbol mapping ===
symbol_map = {
    "VESSEL - As Planned JUB Mooring Plan Installation": "As Planned JUB Mooring Installation",
    "VESSEL - As Planned JUB Mooring Plan Standoff":     "As Planned JUB Mooring Standoff",
}

# === Update SYMBOL field from LAYER ===
with arcpy.da.UpdateCursor(fc, ["LAYER", "SYMBOL"]) as cursor:
    for row in cursor:
        original = row[1]
        row[1] = row[0]  # Default: SYMBOL = LAYER
        # row[1] = symbol_map.get(row[1], row[1])  # Override if match found
        #if row[1] != original:
        #    print(f"🎨 SYMBOL updated: '{original}' ➝ '{row[1]}'")
        cursor.updateRow(row)

print("✅ Section 2 complete: SYMBOL field updated.")


✅ **Section 3: Delete the *Entity* Field**

In [ ]:
# Field name to delete
field_to_delete = "Entity"

# Get a list of field names
field_names = [field.name for field in arcpy.ListFields(fc)]

# Check if the field exists and delete it
if field_to_delete in field_names:
    arcpy.DeleteField_management(fc, field_to_delete)
    print(f"🗑️ Field '*{field_to_delete}*' deleted from *{fc}*.")
else:
    print(f"ℹ️ Field '*{field_to_delete}*' does not exist in *{fc}*.")

#### 🧩 Support Function: Delete a Field from a Feature Class (ArcPy)

In [ ]:
# arcpy.DeleteField_management(fc, ["test"])
# print("🗑️ Field 'SYMBOL' deleted.")

#### 🧩 Support Function: Attribute Table- Remove the first 8 characters}

### 7)  AutoCAD Update


🛠️ AutoCAD Update Instructions
 General Guidelines  
- Every new layer must include a **Layer** field in the attribute table to ensure compatibility between **ArcGIS Pro and AutoCAD**.  
- Layer names in the attribute table should **match the entries** in the **Excel log file** for traceability and consistency.

---
 🔹 Preparing the DWG for Export

1. **Open the DWG file** and delete existing layers that will be replaced.  
   - Use the AutoCAD command:  
     LAYDEL → Enter → Type N → Enter → Select layers by name
     
   - ⚠️ **Warning**: LAYDEL permanently deletes the selected layers and all objects on them. Use with caution.

   **Example layers to delete:**
   - ANNOTATION - As Built Footprint - MPI Adventure`
   - ANNOTATION - As Built Monopile Location`
   - VESSEL - As Built Footprint - MPI Adventure`
   - AS BUILT - Monopile Location`
   - AS BUILT - Monopile Outer Diameter 8pt6m`
   

2. **Save the DWG file** after cleanup.

---

 🔹 Exporting Updated Data from ArcGIS Pro

3. **Open the ArcGIS project MAP:* 005356185-{new version} THOR Master Construction Drawing CAD* that contains the new





4. Use the **Export to CAD** geoprocessing tool:
   - Ensure only required layers are selected for export.Layers are new in ArcGIS Pro.
 

5. **Remove non-essential layers** that should not be included in the export:
   - For example:  
     - `Bathymetry` (use contours instead of raster for CAD exports)

6. Use the **Append** option to add data to the existing DWG file without overwriting unrelated content.

7. Export the **.aprx** as a **Project Package**, ensuring that excluded `Bathymetry` are not part of the package.




#### CAD Tips 
🔍 Zoom to Layer – Quick Steps
- Type QSELECT → Enter
     -Set Property = Layer, and choose your layer
     - Click OK to select all objects on that layer

- Zoom to Selection:
    -Type Z → EnteR
    - Type O (for Object) → Enter



### 8) Portal Update Instructions

📁 Open ArcGIS Project
*\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\_MCD\005356185_THOR_Master_Construction_Drawing.aprx*

🗺️ Open the Map:
- 005356185 THOR Master Construction Drawing Portal

📂 Manage Versions:
- Move the previous version to the Approved group
- Move the new version to the Under Review group

➕ Add New Layers
- Ensure all new or updated data layers are added to the map

♻️ Overwrite Existing Map
- Overwrite: 005356185 THOR Master Construction Drawing Portal

### 9) 🖨️🗺️ Export Maps to the PDFs

- **Copy and paste** previous layout in ArcGIS Pro project:  005356185_THOR_Master_Construction_Drawing.aprx
- 🆕 **Update title** with new revision number
- 📊 **Update table** elements as required
- Edit in the code:  
    - *text_element = layout.listElements("TEXT_ELEMENT", "Text 27 - Revision Box - 01 Checked Initials **8**")[0]*
- 💻 **Run the script** to export updated map books

In [1]:
import arcpy
import PyPDF2

# Define the path to your ArcGIS Pro project
new_version = "11"
aprx_path = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\_MCD\005356185_THOR_Master_Construction_Drawing.aprx"

# Load the project
aprx = arcpy.mp.ArcGISProject(aprx_path)

# Specify the layout name
layout_name = f"005356185-{new_version} THOR Master Construction Drawing"

# Check if the specified layout exists
layout = None
for lyr in aprx.listLayouts():
    if lyr.name == layout_name:
        layout = lyr
        break

if layout is None:
    raise ValueError(f"Layout '{layout_name}' not found in the project.")

# Access the map series
map_series = layout.mapSeries

# Access the map frame to control layers
map_frame = layout.listElements("MAPFRAME_ELEMENT", "Main Map Map Frame")[0]
map_ = map_frame.map

# List all layers in the map to verify the layer name
layers = map_.listLayers()
print("Available layers:")
for layer in layers:
    print(layer.name)

# Access the map series layer "DDP"
ddp_layer = None
for layer in layers:
    if layer.name == "DDP":
        ddp_layer = layer
        break

if ddp_layer is None:
    raise ValueError("Layer 'DDP' not found in the map.")

# Define the output directory for the map books
output_dir = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\_MCD"

# Define the definition queries for each map book
definition_queries = ["SCOPE = 'EEC'", "SCOPE = 'EAC'", "SCOPE = 'FOU'", "SCOPE = 'EOS'"]

# Define the text to update for each map book
approved_by_texts = ["DS", "AM", "RC/JC/OW","NK"]

# Define the output file names for each map book
output_files = ["005356185-11 THOR Master Construction Drawing - EEC.pdf",
                "005356185-11 THOR Master Construction Drawing - EAC.pdf",
                "005356185-11 THOR Master Construction Drawing - FOU.pdf",
                "005356185-11 THOR Master Construction Drawing - EOS.pdf"]

# Layers to be turned on for the first page only
first_page_layers = ["World Imagery (Clarity)", "Nearshore Bathymetry - 004896852-02-FUGG - THOR 2023 LIDAR SURVEY REPORT", "Hillshade Nearshore"]

# Export the first map book with layers on for the first page
for i, definition_query in enumerate(definition_queries):
    # Set the definition query for the map series layer
    ddp_layer.definitionQuery = definition_query
    
    # Update the text element
    text_element = layout.listElements("TEXT_ELEMENT", "Text 27 - Revision Box - 01 Checked Initials 8")[0]
    text_element.text = approved_by_texts[i]
    
    if i == 0:
        # Turn on the specified layers for the first page of the first map book
        for layer in layers:
            if layer.name in first_page_layers:
                layer.visible = True
        
        # Export the first page of the first map book
        first_page_path = output_dir + f"\\{output_files[i].replace('.pdf', '_first_page.pdf')}"
        map_series.exportToPDF(first_page_path, multiple_files="PDF_SINGLE_FILE", resolution=100, page_range_type="RANGE", page_range_string="1")
        
        # Turn off the specified layers for the rest of the pages
        for layer in layers:
            if layer.name in first_page_layers:
                layer.visible = False
        
        # Export the rest of the pages of the first map book (pages 2 to 35)
        remaining_pages_path = output_dir + f"\\{output_files[i].replace('.pdf', '_remaining_pages.pdf')}"
        map_series.exportToPDF(remaining_pages_path, multiple_files="PDF_SINGLE_FILE", resolution=100, page_range_type="RANGE", page_range_string="2-35")
        
        # Merge the first page and the remaining pages into a single PDF
        merger = PyPDF2.PdfFileMerger()
        merger.append(first_page_path)
        merger.append(remaining_pages_path)
        merger.write(output_dir + f"\\{output_files[i]}")
        merger.close()
    else:
        # Ensure the specified layers are off for the other map books
        for layer in layers:
            if layer.name in first_page_layers:
                layer.visible = False
        
        # Export the entire map book
        map_series.exportToPDF(output_dir + f"\\{output_files[i]}", multiple_files="PDF_SINGLE_FILE", resolution=100)

print("Map books exported successfully with updated text elements and 100 DPI resolution!")

Available layers:
DK_THO_AsBuilt_Monopile_pt_UTM32N_pt_v0_Annotation
Class 1
As Built Monopile
DDP
ALARP
ALARP Certified Working Area Anchor Placements
ALARP Certified Working Area
ALARP Certified Working Area OSS
ALARP Certified Working Area Wet Store
WINDFARM
Project Development Area
WTG
WTG Layout 
External Work Platform Heading (°)
External Work Platform (Layout)
Military Radar Platform
Scour protection (22m) v7 
VESSEL
 As Built Footprints - MPI Adventure Annotation
Class 1
As Built Footprints - MPI Adventure
Vessel Ghost Positioning Footprint - Brave Tern
Vessel Ghost Positioning Sectors - MPI Adventure
Vessel Ghost Positioning Sectors - Brave Tern
EOS
EOS Plot Layout - Main Deck
EOS Plot Layout - Bellow Cable Deck
EOS Centroid and pile center coordinates
EOS Cable Exclusion Zone
EEC
Kilometer Points
As Laid EEC North
As Laid EEC South
Offshore Export Cable North
Offshore Export Cable South
Export Cable Corridor (DEA 2019)
Offshore Export Cable ±10m Buffer Boulder Clearance Area
